### Data Extraction from Conference Exit Survey Comments ###

In [ ]:
import pandas as pd
import json

In [ ]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

client = openai.OpenAI()

We will retrieve the comments from a .csv file and store as a list.

In [ ]:
import random

comments_df = pd.read_csv("Conference_Exit_Comments.csv")
comments = comments_df['Comment'].tolist()
random.sample(comments, 1)

Note the addition of response_format = {"type" : "json_object"} to the chat.completions.create() function call

In [ ]:
def analyze_comment(comment):
    prompt = f""" 
    {comment}
    Analyse the comment above and return a JSON array as the result.
    The comments are made by attendees of a marketing conference.
    Extract the following information:

    sentiment: Sentiment of the comment
    
    previous_attendee:  whether or not the comment writer has attended past Apogee conferences
    Answer True if yes, False if no.  Answer Unknown if it is not possible to determine from the comment text.

    VIP:  whether or not the comment writer had VIP access to any of the events
    Answer True if yes, False if no.  Answer Unknown if it is not possible to determine from the comment text.

    planning_to_return:  whether or not the comment writer is planning to return in future years
    Answer True if yes, False if no.  Answer Unknown if it is not possible to determine from the comment text.

    speakers: Extract any speakers that are mentioned by name and output them as a comma separated Python list.

    Format the output as JSON with the following keys:
    sentiment
    previous_attendee
    VIP
    planning_to_return
    speakers
    """
    
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You are a comment analysis tool."},
            {"role": "user", "content": prompt}],
        temperature = 0,
        response_format={ "type": "json_object" }
    )

    response = response.choices[0].message.content
    
    return json.loads(response) 


In [ ]:
comments[3]

In [ ]:
response = analyze_comment(comments[3])
print(response)



In [ ]:
type(response)

In [ ]:
response['sentiment']

In [ ]:
response_accum = []

for comment in comments:
    response = analyze_comment(comment)
    response_accum.append(response)    

In [ ]:
response_accum

In [ ]:
df = pd.DataFrame(response_accum)
df.to_csv('reponses_output.csv', index=False, header=True)